In [1]:
# Dynamic path setup
import sys
from pathlib import Path

# Find project root dynamically
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir

# Add utils to path
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))

# Import the function from the module (make sure the function exists in the file)
from build_training_set import build_training_set

# Test
df = build_training_set(
    train_start="2025-01-01 00:00:00",
    train_end="2025-03-15 11:00:00",
    run_date="2025-03-15 12:00:00"
)

print(f"Shape: {df.shape}")
print(f"Price nulls: {df['Price'].isnull().sum()}/{len(df)}")
print(f"Date range: {df['target_datetime'].min()} to {df['target_datetime'].max()}")
print(df.head(10))

2025-05-26 17:25:37,131 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-26 17:25:37,131 - build_training_set - INFO - 🧠 Observational data van 2025-01-01 00:00:00+00:00 t/m 2025-03-15 12:00:00+00:00
2025-05-26 17:25:37,132 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 12:00:00+00:00, target range: 2025-03-15 12:00:00+00:00 → 2025-03-22 23:00:00+00:00
2025-05-26 17:25:37,159 - build_training_set - INFO - ✅ Observational data geladen: 1765 rijen
2025-05-26 17:25:37,315 - build_training_set - INFO - 🧐 Forecast rows: (27648, 27)
2025-05-26 17:25:37,316 - build_training_set - INFO - 🧐 target_datetime min: 2025-01-01 00:00:00+00:00 | max: 2025-05-31 23:00:00+00:00
2025-05-26 17:25:37,318 - build_training_set - INFO - 🎯 Geselecteerde forecast rijen: 0
2025-05-26 17:25:37,321 - build_training_set - INFO - ✅ Added actual prices to 180 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/bui

Shape: (1765, 31)
Price nulls: 0/1765
Date range: 2025-01-01 00:00:00+00:00 to 2025-03-15 12:00:00+00:00
     Price           target_datetime      Load  shortwave_radiation  \
0  0.01362 2025-01-01 00:00:00+00:00  12049.25                  0.0   
1  0.00624 2025-01-01 01:00:00+00:00  11957.50                  0.0   
2  0.00416 2025-01-01 02:00:00+00:00  11636.25                  0.0   
3  0.00328 2025-01-01 03:00:00+00:00  11310.50                  0.0   
4  0.00068 2025-01-01 04:00:00+00:00  11135.25                  0.0   
5  0.00000 2025-01-01 05:00:00+00:00  11185.75                  0.0   
6  0.00076 2025-01-01 06:00:00+00:00  11385.00                  0.0   
7  0.00079 2025-01-01 07:00:00+00:00  11695.25                  0.0   
8  0.00189 2025-01-01 08:00:00+00:00  12041.50                  0.0   
9  0.00750 2025-01-01 09:00:00+00:00  12485.75                  0.0   

   temperature_2m  direct_normal_irradiance  diffuse_radiation  Flow_NO  \
0             0.0                     

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Test parameters
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00" 
base_run = "2025-03-15 00:00:00"

# Forecast config
forecast_horizon = 168  # 168 hours = 7 days

# Storage for results
rmse_results = []

print("🔍 Testing XGBoost - RMSE per forecast day")
print("=" * 60)

for i in range(30):  # 30 rolling windows
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        # Get training set
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is not None and len(df) > 0:
            # Convert datetime
            df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
            df = df.sort_values('target_datetime')
            df = df.set_index('target_datetime')

            # Define features
            all_features = [
                 'Load','shortwave_radiation','temperature_2m','direct_normal_irradiance','diffuse_radiation','Flow_NO','yearday_cos','Flow_GB',
                 'month','is_dst','yearday_sin','wind_speed_10m','is_non_working_day','hour_cos','is_weekend','cloud_cover','weekday_sin','hour_sin','weekday_cos'
            ]
            target = 'Price'

            train_cutoff = pd.Timestamp(end, tz="UTC")
            forecast_data = df[df.index > train_cutoff].copy()
            train_data = df[df.index <= train_cutoff].copy()

            # Clean forecast set
            if len(forecast_data) >= forecast_horizon and forecast_data['Price'].notna().sum() > 0:
                forecast_data = forecast_data.iloc[:forecast_horizon]  # Limit to 168h
                X_train = train_data[all_features]
                y_train = train_data[target]
                X_test = forecast_data[all_features].copy()
                y_test = forecast_data[target]

                # Train model
                model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
                model.fit(X_train, y_train)

                # Predict
                y_pred = model.predict(X_test)

                # Evaluate per day
                valid_data = forecast_data.copy()
                valid_data['y_true'] = y_test
                valid_data['y_pred'] = y_pred
                valid_data['forecast_hour'] = range(len(valid_data))
                valid_data['forecast_day'] = (valid_data['forecast_hour'] // 24) + 1

                day_rmses = {}
                for day in range(1, 8):
                    day_data = valid_data[valid_data['forecast_day'] == day]
                    if len(day_data) > 0:
                        rmse = np.sqrt(mean_squared_error(day_data['y_true'], day_data['y_pred']))
                        day_rmses[f'Day_{day}'] = rmse
                    else:
                        day_rmses[f'Day_{day}'] = np.nan

                result = {
                    'iteration': i + 1,
                    'run_date': run.strftime('%Y-%m-%d'),
                    'valid_predictions': len(valid_data),
                    **day_rmses
                }
                rmse_results.append(result)
                print(f"Day {i+1}: ✅ {len(valid_data)} predictions, Run: {run.strftime('%m-%d')}")
            else:
                print(f"Day {i+1}: ❌ No forecast data with actual prices")
        else:
            print(f"Day {i+1}: ❌ No training data")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create RMSE matrix
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 RMSE MATRIX - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {len(rmse_df)}/30")

    day_columns = [f'Day_{i}' for i in range(1, 8)]
    available_day_cols = [col for col in day_columns if col in rmse_df.columns]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
color_pal = sns.color_palette()
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
import sqlite3

conn = sqlite3.connect('../data/WARP.db')
df = pd.read_sql_query("SELECT * FROM training_set", conn)
conn.close()
# change datetime to index
df.set_index('target_datetime', inplace=True)
# convert to datetime
df.index = pd.to_datetime(df.index)
print(df.dtypes)


In [ ]:
df['Price'].plot(kind='hist', bins=500)
plt.ylim(top=120)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Price Distribution')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = [
    'apparent_temperature',
    'temperature_2m',
    'direct_normal_irradiance',
    'diffuse_radiation',
    'yearday_sin',
    'Flow_BE',
    'hour_sin',
    'is_non_working_day',
    'is_dst',
    'is_weekend',
    'is_holiday',
    'weekday_cos',
    'wind_speed_10m',
    'hour_cos',
    'weekday_sin',
    'cloud_cover',
    'Flow_GB',
    'yearday_cos',
    'Flow_NO',
    'Load'
]
target = 'Price'

# Safe datetime handling
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.set_index('datetime')
else:
    print("'datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()

# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-14 12:00", tz='UTC')
lag = timedelta(hours=36)
forecast_horizon = timedelta(hours=144)

# Store RMSEs
rmses = []

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag # check current time  
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, RMSE: {rmse:.3f}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.3f}")

In [ ]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import datetime

# Configuration
MODEL_CODE = "XGB_1fold_split"
PARAMETERS_USED = {
    "temp_2m": True,
    "shortwave_radiation": True,
    "windspeed_10m": True
}
timestamp_predict = pd.to_datetime("2025-03-15 00:00:00", utc=True)
TARGET = 'Price'

# Define features
COMMON_FEATURES = [
    'yearday_cos', 'yearday_sin', 'month',
    'shortwave_radiation', 'windspeed_10m', 'apparent_temperature', 'temperature_2m',
    'direct_normal_irradiance', 'diffuse_radiation',
    'cloud_cover', , 'hour_cos', 'hour_sin', 'is_non_working_day',
    'weekday_sin', 'weekday_cos', 'is_holiday',
]

# Add Load only to training features
TRAIN_FEATURES = COMMON_FEATURES + ['Load', 'Flow_NO', 'Flow_GB', 'Flow_BE', 'Wind_Vol', 'Solar_Vol']
TEST_FEATURES = COMMON_FEATURES

# Sort by index
df = df.sort_index()

# Train-test split based on timestamp_predict
train = df[df.index < timestamp_predict]
test = df[df.index >= timestamp_predict]

X_train = train[TRAIN_FEATURES]
y_train = train[TARGET]
X_test = test[TEST_FEATURES]
y_test = test[TARGET]

# Train model
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster='gbtree',
    n_estimators=1200,
    early_stopping_rounds=50,
    objective='reg:squarederror',
    max_depth=3,
    learning_rate=0.02
)

reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=100
)

y_pred = reg.predict(X_test)

# Store predictions
y_test.index = pd.to_datetime(y_test.index)
pred_df = pd.DataFrame({
    'datetime': y_test.index,
    'y_true': y_test.values,
    'y_pred': y_pred
})

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"1-Fold RMSE: {rmse:.4f}")

# Collect results
results = []
for _, row in pred_df.iterrows():
    results.append({
        'timestamp_predict': timestamp_predict.strftime('%d-%m-%Y %H:%M'),
        'datetime': row['datetime'].strftime('%d-%m-%Y %H:%M'),
        'model_code': MODEL_CODE,
        'price': round(row['y_pred'], 4),
        'true_price': round(row['y_true'], 4),
        'RSME': round(rmse, 4),
        'fold': 1,
        'parameters(JSON)': json.dumps(PARAMETERS_USED)
    })

# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("model_results_log.csv", mode='a', index=False)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = ['hour_cos', 'Load', 'hour_sin', 'weekday_sin', 'weekday_cos', 'Solar_Vol', 'Wind_Vol',
            'WindOffshore_Vol', 'is_holiday','Total_Flow']

target = 'Price'

# Safe datetime handling
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.set_index('datetime')
else:
    print("'datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()

# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-05 12:00", tz='UTC')
lag = timedelta(hours=36)
forecast_horizon = timedelta(hours=144)

# Store RMSEs
rmses = []

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, RMSE: {rmse:.3f}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.3f}")

In [ ]:
# Extract the forecast origin dates and RMSEs from the previous loop
forecast_origins = []
rmses_per_run = []

start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-05 12:00", tz='UTC')
num_runs = len(rmses)
current_time = start_date

for i in range(num_runs):
    forecast_origins.append(current_time)
    current_time += timedelta(days=1)

# Plot RMSE vs. forecast origin date
plt.figure(figsize=(15, 5))
plt.plot(forecast_origins, rmses, marker='o', linestyle='-', color=color_pal[0])
plt.title('RMSE vs. First Predicted Date (per run)')
plt.xlabel('Forecast Origin (First Predicted Date)')
plt.ylabel('RMSE')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Calculate RMSE per day (averaged over all runs where the day is included in the forecast horizon)

# Assume forecast_origins and rmses are available from previous cells
# Each run predicts 6 days (144 hours), so for each run, map RMSE to each predicted day

# Build a DataFrame mapping each forecast run to its predicted days
forecast_horizon_days = 6
forecast_origin_dates = pd.to_datetime(forecast_origins)
rmse_per_day = {}

for run_idx, origin in enumerate(forecast_origin_dates):
    for day_offset in range(forecast_horizon_days):
        day = (origin + pd.Timedelta(hours=36) + pd.Timedelta(days=day_offset)).normalize()
        if day not in rmse_per_day:
            rmse_per_day[day] = []
        rmse_per_day[day].append(rmses[run_idx])

# Compute average RMSE per day
avg_rmse_per_day = pd.Series({day: np.mean(vals) for day, vals in rmse_per_day.items()})
avg_rmse_per_day = avg_rmse_per_day.sort_index()

# Plot
plt.figure(figsize=(15, 5))
plt.plot(avg_rmse_per_day.index, avg_rmse_per_day.values, marker='o', linestyle='-', color=color_pal[1])
plt.title('Average RMSE per Predicted Day (Averaged over all runs)')
plt.xlabel('Predicted Day')
plt.ylabel('Average RMSE')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Get feature importances from the last trained model
importances = model.feature_importances_
feature_names = model.feature_names_in_

# Create a DataFrame for better visualization
feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feat_imp_df)

# Optional: Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette='viridis')
plt.title('Feature Importance (XGBoost)')
plt.tight_layout()
plt.show()